In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objs as go

In [2]:
ticker_symbol = 'BTC-USD'
period = '1y'

data = yf.download(ticker_symbol, period=period)

[*********************100%%**********************]  1 of 1 completed


In [3]:
def MACD(series, n_fast=12, n_slow=26, signal=9):
    fast_ema = series.ewm(span=n_fast, adjust=False).mean()
    slow_ema = series.ewm(span=n_slow, adjust=False).mean()
    macd = fast_ema - slow_ema
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    return macd, signal_line

def RSI(series, period=14):
    delta = series.diff()
    gain = delta.where(delta > 0, 0).rolling(window=period).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

In [6]:
class Backtest:
    def __init__(self, data, cash=10000, commission=0.002):
        self.data = data
        self.initial_cash = cash
        self.cash = cash
        self.commission = commission
        self.position = 0
        self.trades = []
        self.equity = []

    def buy(self, price):
        if self.position == 0 and self.cash > 0:
            invest_amount = self.cash * 0.9  # 90% of the cash
            commission_paid = invest_amount * self.commission
            amount_invested = invest_amount - commission_paid
            self.position = amount_invested / price
            self.cash -= invest_amount
            self.trades.append(('buy', price, commission_paid))

    def sell(self, price):
        if self.position > 0:
            proceeds = self.position * price
            commission_paid = proceeds * self.commission
            amount_received = proceeds - commission_paid
            self.cash += amount_received
            self.position = 0
            self.trades.append(('sell', price, commission_paid))

    def run(self):
        self.data['MACD'], self.data['Signal'] = MACD(self.data['Close'])
        self.data['RSI'] = RSI(self.data['Close'])

        for i in range(1, len(self.data)):
            self.equity.append(self.cash + self.position * self.data['Close'][i])
            if self.data['MACD'][i] > self.data['Signal'][i] and self.data['RSI'][i] < 30:
                self.buy(self.data['Close'][i])
            elif self.data['MACD'][i] < self.data['Signal'][i] or self.data['RSI'][i] > 70:
                self.sell(self.data['Close'][i])

    def results(self):
        final_value = self.cash + self.position * self.data['Close'].iloc[-1]
        return {
            'final_cash': final_value,
            'total_trades': len(self.trades),
            'net_profit': final_value - self.initial_cash,
            'total_commission_paid': sum(trade[2] for trade in self.trades)
        }

    def plot_results(self):
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=self.data.index, y=self.equity, name='Equity Curve'))

        buy_trades = [self.data.index[i] for i, trade in enumerate(self.trades) if trade[0] == 'buy']
        sell_trades = [self.data.index[i] for i, trade in enumerate(self.trades) if trade[0] == 'sell']

        fig.add_trace(go.Scatter(x=buy_trades, y=[self.data['Close'][i] for i in buy_trades],
                                mode='markers', name='Buy', marker_symbol='triangle-up', marker_color='green'))

        fig.add_trace(go.Scatter(x=sell_trades, y=[self.data['Close'][i] for i in sell_trades],
                                mode='markers', name='Sell', marker_symbol='triangle-down', marker_color='red'))

        # Convert 12x8 inches to pixels
        width_px = 12 * 96  # 12 inches in pixels
        height_px = 8 * 96  # 8 inches in pixels

        fig.update_layout(
            title='Backtest Results',
            xaxis_title='Date',
            yaxis_title='Price',
            legend_title='Legend',
            width=width_px,
            height=height_px
        )
        fig.show()

In [7]:
bt = Backtest(data)
bt.run()
print(bt.results())
bt.plot_results()

{'final_cash': 10693.33096969336, 'total_trades': 2, 'net_profit': 693.3309696933593, 'total_commission_paid': 37.42551296531735}


C:\Users\chris_pedretti\AppData\Local\Temp\ipykernel_20752\3366745386.py:34: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\chris_pedretti\AppData\Local\Temp\ipykernel_20752\3366745386.py:35: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\chris_pedretti\AppData\Local\Temp\ipykernel_20752\3366745386.py:37: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\chris_pedretti\AppData\Local\Temp\ipykernel_20752\33667453